In [2]:
%%capture

!pip install bs4
!pip install nbformat
!pip install pandas
!pip install geopandas
!pip install requests
!pip install --upgrade xlrd
!pip install openpyxl

In [3]:
%%capture

%run ./Tidy.ipynb

# Test d'un autre modèle: faire du clustering 

On veut ici utiliser la masse d'observation dont on dispose avec la prévalence pour toutes les drogues et toutes les classes d'âge.

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn.cluster import KMeans  # pour kmeans
import seaborn as sns  # pour scatterplots

X_vars = ['Prevalence (%)','Drug']

# Une première idée rejetée

Nous avons en premier lieu essayé de prédire la légalisation ou non du cannabis au sein d'un même pays à l'aide d'un SVM prenant en entrée la prévalence dans chaque pays.

Cette première approche n'a pas porté les fruits escomptés et ce pour plusieurs raisons, la principale étant une inadéquation entre notre jeu de données (trop faible) et le modèle choisi. Cette dernière section nous permet d'illustrer les raisons de cet échec et de tirer des conclusions sur la démarche à adopter lors du choix et de la mise en place de modèles.

### Sélection des données

On s'était restreint dans le cadre de la prédiction de la légalisation ou non du cannabis dans chaque pays, au pourcentage d'individus ayant déjà consommé du cannabis dans leur vie pour chaque pays.
On se rend alors vite compte que cette restriction nous donne un jeu de données très faible (1 observation par pays, soit seulement 29 observations). Ce qui va poser problème par la suite.

In [5]:
import pandas as pd

# On récupère la table de légalisation que l'on a nettoyé en amont

legal = legal_pays_clean

# On prend la prévalence sur la vie entière

df = pd.read_csv('https://raw.githubusercontent.com/refouch/Projet_Python_2A/refs/heads/main/Data/EUDA/edr2024-gps-current-table-1.csv')

dfcurrent = df[df['Substance'] == 'Cannabis']

dfcurrentadults = dfcurrent[dfcurrent['Age'] == 'All adults (15-64)']

dfcurrentadultslf = dfcurrentadults[dfcurrentadults['Recall period'] == 'Lifetime']

# On effectue un merge les deux tables
dfadultslf_legal = dfcurrentadultslf.join(legal, on = 'Country')

In [6]:
# Maintenant on tente d'entraîner le modèle parce que psartek.

from sklearn import svm
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

# Recodage des variables catégorielles

dfadultslf_legal['Recreational_num'] = dfadultslf_legal['Recreational'].replace({'Decriminalized' : 2, 'Legal' : 1, 'Illegal' : 0})
dfadultslf_legal['Medical_num'] = dfadultslf_legal['Medical'].replace({'Decriminalized' : 2, 'Legal' : 1, 'Illegal' : 0})

X = dfadultslf_legal[['Prevalence (%)','Males (%)','Females (%)']]
Y = dfadultslf_legal['Recreational_num']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)

# Création du classifieur SVM avec le paramètre C=1
svm_classifier = svm.SVC(C=1, kernel='linear')

# Entraînement du modèle sur l'ensemble d'estimation
svm_classifier.fit(X_train, Y_train)

# Prédictions sur l'ensemble de test
Y_pred = svm_classifier.predict(X_test)

# Accuracy score = Normalization on correctly predicted outputs
accuracy_score = sklearn.metrics.accuracy_score(Y_test, Y_pred)
print(accuracy_score)

# Precision score = ratio tp / (tp + fp) -> ratio de vrais positifs contre faux positifs
precision_score = sklearn.metrics.precision_score(Y_test, Y_pred, average= 'micro')
print(precision_score)

#Recall 
recall_score = sklearn.metrics.recall_score(Y_test, Y_pred, average= 'micro')
print(recall_score)

#F1
f1_score = sklearn.metrics.f1_score(Y_test, Y_pred, average= 'micro')
print(f1_score)

0.6666666666666666
0.6666666666666666
0.6666666666666666
0.6666666666666666


/tmp/ipykernel_166200/185401596.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfadultslf_legal['Recreational_num'] = dfadultslf_legal['Recreational'].replace({'Decriminalized' : 2, 'Legal' : 1, 'Illegal' : 0})
/tmp/ipykernel_166200/185401596.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfadultslf_legal['Medical_num'] = dfadultslf_legal['Medical'].replace({'Decriminalized' : 2, 'Legal' : 1, 'Illegal' : 0})


# Encore des test: avec la table de lelay


In [7]:
def get_latest_data(df, new_df, variable):
    """ Parcours la table pour créer une nouvelle variable qui prend en compte les données les plus récentes disponibles
        Ce afin de pallier aux fait qu'on manque de données récentes pour certaines variables"""

    columns = list(df.columns.values)
    good_col = []

    new_df['Country'] = df['Country']

    print(columns)

    for i in range(len(columns)):
        if variable in columns[i]:
            good_col.append(columns[i])

    for index in df.index:
        for column in good_col:
            if pd.isna(df.at[index, column]) == False:
                new_df.at[index, variable] = df.at[index, column]
                break

    new_df.dropna(inplace = True)

    return new_df

In [8]:
import pandas as pd

# On récupère la table de légalisation que l'on a nettoyé en amont

legal = legal_pays_clean

# On prend la prévalence sur la vie entière

dfcurrent = test

df_conso = pd.DataFrame()

df_conso = get_latest_data(dfcurrent,df_conso,'cannabis.conso_x')
df_conso = get_latest_data(dfcurrent,df_conso,'cannabis.conso_y')
df_conso = get_latest_data(dfcurrent,df_conso,'cannabis.prod')
df_conso = get_latest_data(dfcurrent,df_conso,'cannabis.ventes')


#On effectue un merge les deux tables
dfadultslf_legal = df_conso.join(legal, on = 'Country')

dfadultslf_legal.dropna(inplace =True)

dfadultslf_legal

['Country', '2022.0.cannabis.conso_x', '2021.0.cannabis.conso_x', '2020.0.cannabis.conso_x', '2019.0.cannabis.conso_x', '2018.0.cannabis.conso_x', '2017.0.cannabis.conso_x', '2016.0.cannabis.conso_x', '2015.0.cannabis.conso_x', '2014.0.cannabis.conso_x', '2013.0.cannabis.conso_x', '2012.0.cannabis.conso_x', '2011.0.cannabis.conso_x', '2010.0.cannabis.conso_x', '2009.0.cannabis.conso_x', '2008.0.cannabis.conso_x', '2007.0.cannabis.conso_x', '2022.0.herbe.conso', '2021.0.herbe.conso', '2020.0.herbe.conso', '2019.0.herbe.conso', '2018.0.herbe.conso', '2017.0.herbe.conso', '2016.0.herbe.conso', '2015.0.herbe.conso', '2014.0.herbe.conso', '2013.0.herbe.conso', '2012.0.herbe.conso', '2011.0.herbe.conso', '2010.0.herbe.conso', '2009.0.herbe.conso', '2008.0.herbe.conso', '2007.0.herbe.conso', '2022.0.huile.conso', '2021.0.huile.conso', '2020.0.huile.conso', '2019.0.huile.conso', '2018.0.huile.conso', '2017.0.huile.conso', '2016.0.huile.conso', '2015.0.huile.conso', '2014.0.huile.conso', '2013.

,Country,cannabis.conso_x,cannabis.conso_y,cannabis.prod,cannabis.ventes,Recreational,Medical
0,Austria,19909.0,19909.0,2601.0,6784.0,Decriminalized,Legal
1,Belgium,29572.0,29572.0,661.0,5805.0,Decriminalized,Legal
3,Croatia,5159.0,5159.0,252.0,1016.0,Decriminalized,Legal
4,Cyprus,563.0,563.0,18.0,166.0,Illegal,Legal
5,Czechia,6443.0,6443.0,184.0,728.0,Decriminalized,Legal
10,Germany,174876.0,174876.0,494.0,39366.0,Legal,Legal
11,Greece,5764.0,5764.0,537.0,3601.0,Illegal,Legal
12,Hungary,3437.0,3437.0,117.0,587.0,Illegal,Illegal
19,Netherlands,2413.0,2413.0,2245.0,2245.0,Decriminalized,Legal
22,Portugal,7020.0,7020.0,247.0,6112.0,Decriminalized,Legal


In [15]:
# Maintenant on tente d'entraîner le modèle parce que psartek.

from sklearn import svm
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

# Recodage des variables catégorielles

dfadultslf_legal['Recreational_num'] = dfadultslf_legal['Recreational'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})
dfadultslf_legal['Medical_num'] = dfadultslf_legal['Medical'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})

X = dfadultslf_legal[['cannabis.conso_x','cannabis.conso_y','cannabis.prod','cannabis.ventes']]
Y = dfadultslf_legal['Recreational_num']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)

# Création du classifieur SVM avec le paramètre C=1
svm_classifier = svm.SVC(C=1, kernel='linear')

# Entraînement du modèle sur l'ensemble d'estimation
svm_classifier.fit(X_train, Y_train)

# Prédictions sur l'ensemble de test
Y_pred = svm_classifier.predict(X_test)

# Accuracy score = Normalization on correctly predicted outputs
accuracy_score = sklearn.metrics.accuracy_score(Y_test, Y_pred)
print(accuracy_score)

# Precision score = ratio tp / (tp + fp) -> ratio de vrais positifs contre faux positifs
precision_score = sklearn.metrics.precision_score(Y_test, Y_pred, average= 'micro')
print(precision_score)

#Recall 
recall_score = sklearn.metrics.recall_score(Y_test, Y_pred, average= 'micro')
print(recall_score)

#F1
f1_score = sklearn.metrics.f1_score(Y_test, Y_pred, average= 'micro')
print(f1_score)

#psuh

/tmp/ipykernel_166200/258817588.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfadultslf_legal['Recreational_num'] = dfadultslf_legal['Recreational'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})
/tmp/ipykernel_166200/258817588.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfadultslf_legal['Medical_num'] = dfadultslf_legal['Medical'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})


0.0
0.0
0.0
0.0


## Regression logistique sur les même variables juste pour essayer de voir si c'est mieux

In [ ]:
# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression(random_state=16)

# fit the model with data
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_test)

accuracy = sklearn.metrics.accuracy_score(Y_pred,Y_test)

print(accuracy)
#accuracy

0.0
